In [1]:
import pandas as pd;
import json;
import math;

In [2]:
map_json = open('datamap.json');
nav = json.load(map_json)['raw'];

dataset = nav['files'];
pd.DataFrame(dataset).reindex(columns=['name','file','range', 'order'])

,name,file,range,order
0,S&P 500,SPX_WSJ.csv,"{'start': '2/1/1991', 'end': '3/11/2021'}","[Date, Open, High, Low, Close]"
1,SENSEX,SENSEX_WSJ.csv,"{'start': '1/1/1991', 'end': '3/11/2021'}","[Date, Open, High, Low, Close]"
2,NIKKEI 225,N225_WSJ.csv,"{'start': '4/1/1991', 'end': '2/11/2021'}","[Date, Open, High, Low, Close]"
3,FTSE 100,UKX_WSJ.csv,"{'start': '2/1/1991', 'end': '3/11/2021'}","[Date, Open, High, Low, Close]"
4,BTC/USD,BTC-USD_Yahoo.csv,"{'start': '17/9/2014', 'end': '4/11/2021'}","[Date, Open, High, Low, Close, Adj Close, Volume]"


In [3]:
# Load Dataset, Get Date and Closing and Reverse Where needed;
data_snp5 = pd.read_csv('raws/SPX_WSJ.csv')[['Date', 'Close']][::-1];
data_sens = pd.read_csv('raws/SENSEX_WSJ.csv')[['Date', 'Close']][::-1];
data_n225 = pd.read_csv('raws/N225_WSJ.csv')[['Date', 'Close']][::-1];
data_ftse = pd.read_csv('raws/UKX_WSJ.csv')[['Date', 'Close']][::-1];
data_btco = pd.read_csv('raws/BTC-USD_Yahoo.csv')[['Date', 'Close']];

stocks={}
names=['data_snp5', 'data_sens', 'data_n225', 'data_ftse', 'data_btco'];

for i,j in zip([data_snp5, data_sens, data_n225, data_ftse, data_btco],names):
    # Reformat Date string to DateTime Object
    dt = pd.to_datetime(i['Date'], infer_datetime_format=True);
    i['Date'] = dt;
    
    # Reset Index & Resample
    i = i.set_index(['Date']);
    i = i.resample('D').mean().fillna(0);
    # Store in new Object
    stocks[j]=i;

In [4]:
# Replace 0 values with previous close price;
for p in stocks:
    close=stocks[p]['Close'];
    for i in range(len(close)-1):
        if close[i+1]==0:
            close[i+1]=close[i];
    stocks[p]['Close']=close;

In [5]:
# Balance dates to b/w 4 Jan 91 to 1 Nov 21;
for p in stocks:
    stocks[p] = stocks[p].drop(stocks[p][stocks[p].index < pd.Timestamp('1991-01-04')].index);
    stocks[p] = stocks[p].drop(stocks[p][stocks[p].index > pd.Timestamp('2021-11-01')].index);
    
for i in names:
    print(i+"\n", stocks[i].head());

data_snp5
              Close
Date              
1991-01-04  321.00
1991-01-05  321.00
1991-01-06  321.00
1991-01-07  315.44
1991-01-08  314.90
data_sens
               Close
Date               
1991-01-04   982.35
1991-01-05   982.35
1991-01-06   982.35
1991-01-07  1010.77
1991-01-08  1010.77
data_n225
                Close
Date                
1991-01-04  24069.18
1991-01-05  24069.18
1991-01-06  24069.18
1991-01-07  23736.57
1991-01-08  22897.84
data_ftse
              Close
Date              
1991-01-04  2126.1
1991-01-05  2126.1
1991-01-06  2126.1
1991-01-07  2113.3
1991-01-08  2099.9
data_btco
                  Close
Date                  
2014-09-17  457.334015
2014-09-18  424.440002
2014-09-19  394.795990
2014-09-20  408.903992
2014-09-21  398.821014


In [8]:
for i in names:
    stocks[i].to_csv(r"clean/"+i+".csv", header=True);